In [5]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')

# Contexte :
L’analyse massive des données de santé a révolutionné le secteur médical, notamment en oncologie, en améliorant le pronostic et le traitement des maladies complexes comme le cancer. Les modèles prédictifs permettent des décisions thérapeutiques plus précises et adaptées, optimisant ainsi les soins aux patients.

Le **Data Challenge de QRT**, en collaboration avec l’Institut Gustave Roussy, vise à prédire le risque de décès des patients atteints d’une leucémie myéloïde adulte en utilisant des données cliniques et moléculaires. L’objectif est d’optimiser les stratégies thérapeutiques en identifiant les patients à haut ou faible risque, afin d’améliorer leur prise en charge.

Nous allons travailler sur un **jeu de données** comprenant des informations de 3 323 patients en entraînement et 1 193 en test, issues de 24 centres cliniques. Deux types de données sont exploitées :
- **Données cliniques** (ex. taux de globules blancs, hémoglobine, anomalies chromosomiques).
- **Données moléculaires** (mutations génétiques et impact sur les protéines).

L’évaluation repose sur l’**IPCW-C-index**, une métrique adaptée aux données censurées. Un **modèle de risques proportionnels de Cox** sert de benchmark, comparé à un modèle LightGBM plus simple. L’objectif final est de soumettre des prédictions de risque de décès sous forme de fichier CSV.

Dans le cadre du cours de **machine learning**, nous travaillons sur ce projet pour développer des modèles prédictifs et améliorer la compréhension des approches en analyse de survie.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
import lightgbm as lgb
from pycox.models import CoxPH, DeepHitSingle
import torchtuples as tt
import torch

from sksurv.util import Surv

In [13]:
### Chargement des données train
Xtrain_clin = pd.read_csv("C:/Users/omaim/OneDrive/Bureau/Machine Learning/X_test/clinical_test.csv")
Xtrain_mol = pd.read_csv("C:/Users/omaim/OneDrive/Bureau/Machine Learning/X_test/molecular_test.csv")
Y_train = pd.read_csv("C:/Users/omaim/OneDrive/Bureau/Machine Learning/target_train.csv")

### Chargement des données test
Xtest_clin = pd.read_csv("C:/Users/omaim/OneDrive/Bureau/Machine Learning/X_test/clinical_test.csv")
Xtest_mol = pd.read_csv("C:/Users/omaim/OneDrive/Bureau/Machine Learning/X_test/molecular_test.csv")


In [85]:
print("Aperçu des données :")
#print(Xtrain_clin.head())
#print(Xtrain_mol.head())

Aperçu des données :


# Données cliniques : 
- ID : Identifiant unique du patient.
- CENTER : Centre médical où le patient a été traité.
- BM_BLAST : Pourcentage de cellules blastiques dans la moelle osseuse (critère clé pour les leucémies).
- WBC : Nombre de globules blancs (White Blood Cell count).
- ANC : Nombre absolu de neutrophiles.
- MONOCYTES : Nombre de monocytes.
- HB : Niveau d’hémoglobine.
- PLT : Nombre de plaquettes sanguines.
- CYTOGENETICS : Anomalies cytogénétiques détectées.

# Données moléculaires
- ID : Identifiant unique du patient.
- CHR : Chromosome où la mutation a été détectée.
- START / END : Position génomique de la mutation.
- REF / ALT : Base de référence et base mutée.
- GENE : Nom du gène affecté.
- PROTEIN_CHANGE : Modification de la protéine codée.
- EFFECT : Effet de la mutation (non_synonymous_codon, stop_gained, etc.).
- VAF : Fréquence allélique de la mutation (proportion des copies de l’ADN affectées).
- DEPTH : Profondeur de lecture (nombre de fois que cette mutation a été détectée dans le séquençage).

In [15]:
print("Aperçu des données cible:")
#print(Y_train.head())

Aperçu des données cible:


# Variable cible (survie des patiencts)
- ID : 
- OS_YEARS : Durée de survie en années après le diagnostic.
- OS_STATUS : Statut de survie (0 = en vie, 1 = décédé).

In [16]:
print("Description statistique des données :")
#print(Xtrain_clin.describe())
#print(Xtrain_mol.describe())

Description statistique des données :


In [33]:
print("Analyse des valeurs manquantes :")
#print(Xtrain_mol.isna().sum())
#print(Xtrain_clin.isna().sum())

Analyse des valeurs manquantes :


In [34]:
Xtrain_mol.head()
Xtrain_mol["ID"].value_counts()
Xtrain_mol["EFFECT"] = Xtrain_mol["EFFECT"].astype(str)

Chaque ID pourrait représenter un échantillon de patient, et plusieurs entrées pour un même ID indiquent que différents marqueurs génétiques ou mutations ont été analysés dans cet échantillon. Par exemple, chaque ligne pourrait correspondre à une mutation différente observée dans l'échantillon du patient.

 - La fréquence d'allèles variant (VAF) diffère parmi les entrées, indiquant que les niveaux de présence de ces mutations varient. Cela peut être dû à des différences dans l'échantillonnage ou la prévalence de la mutation dans différentes cellules de l'échantillon.
 - Le PROTEIN_CHANGE montre que les modifications protéiques dues aux mutations sont spécifiques à chaque entrée, ce qui souligne l'importance de chaque mutation unique dans l'analyse moléculaire.
 - Les champs START et END indiquent les positions exactes des mutations sur le chromosome, et bien que certaines puissent se chevaucher ou être identiques, les mutations elles-mêmes peuvent être différentes.

In [ ]:
aggregated_data = Xtrain_mol.groupby('ID').agg({
    'VAF': 'mean',  # Moyenne des fréquences d'allèles variant
    'DEPTH': 'sum',  # Somme des profondeurs de séquençage
    'EFFECT': lambda x: ', '.join(x.unique())  # Liste unique des effets
}).reset_index()

In [36]:
Xtrain_mol1 = Xtrain_mol.drop_duplicates(subset="ID", keep = False)
Xtest_mol = Xtest_mol.drop_duplicates(subset="ID", keep=False)

In [37]:
Xtrain_mol1["ID"].value_counts()

ID
KYW1048    1
KYW135     1
KYW430     1
KYW1076    1
KYW560     1
          ..
KYW746     1
KYW235     1
KYW969     1
KYW918     1
KYW18      1
Name: count, Length: 257, dtype: int64

In [38]:
#Merge la data VERIFIER NB ROWS PAR DATAFRAME
data_train = Xtrain_clin.merge(Xtrain_mol, on='ID', how = "outer")
X_test = Xtest_clin.merge(Xtest_mol, on='ID', how='outer')

In [39]:
data_train.head()

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH
0,KYW1,KYW,68.0,3.45,0.5865,NaN,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",2,25467449.0,25467449.0,C,A,DNMT3A,p.G543C,non_synonymous_codon,0.3840,799.0
1,KYW1,KYW,68.0,3.45,0.5865,NaN,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",5,170837543.0,170837543.0,-,TCTG,NPM1,p.L287fs,frameshift_variant,0.2100,257.0
2,KYW1,KYW,68.0,3.45,0.5865,NaN,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",X,154301677.0,154301677.0,G,T,BRCC3,p.G56X,stop_gained,0.0270,586.0
3,KYW1,KYW,68.0,3.45,0.5865,NaN,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",NaN,NaN,NaN,NaN,NaN,FLT3,FLT3_ITD,ITD,0.3639,NaN
4,KYW10,KYW,1.0,1.61,0.6118,NaN,9.9,85.0,"47,XY,+8",20,57484421.0,57484421.0,G,A,GNAS,p.R201H,non_synonymous_codon,0.2580,508.0


In [40]:
data_train = data_train.drop(columns = ["CENTER"])
X_test = X_test.drop(columns=["CENTER"], errors='ignore')

In [41]:
#Fonction pour préparation de données
def clean_data(data, columns):
    data_cleaned = data.dropna(subset = columns)
    return data_cleaned

In [87]:
target_data = Y_train.dropna()
target_data['OS_YEARS'] = pd.to_numeric(target_data['OS_YEARS'], errors='coerce')
target_data['OS_STATUS'] = target_data['OS_STATUS'].astype(bool)
target_data

,ID,OS_YEARS,OS_STATUS
0,P132697,1.115068,True
1,P132698,4.928767,False
2,P116889,2.043836,False
3,P132699,2.476712,True
4,P132700,3.145205,False
...,...,...,...
3316,P121826,0.547945,False
3317,P121827,2.339726,False
3320,P121830,1.997260,False
3321,P121853,0.095890,True


In [43]:
#il faut regrouper en personnes ayant vécu 0 à 2 ans de 2ans à 4 ...
En_vie = target_data[target_data["OS_STATUS"]==1.0]
Mort = target_data[target_data["OS_STATUS"]==0.0]

En_vie = En_vie.drop(columns = ["OS_STATUS"])
Mort = Mort.drop(columns = ["OS_STATUS"])


In [44]:
#Distribution de la variable cible
#plt.figure(figsize=(10, 5))
#sns.histplot(Mort, kde=True)
#plt.title("Distribution de la variable cible pour personnes décédées")
#plt.show()


In [45]:
#Distribution de la variable cible
#plt.figure(figsize=(10, 5))
#sns.histplot(En_vie, kde=True)
#plt.title("Distribution de la variable cible pour personnes en vie")
#plt.show()

In [47]:
data_train.CYTOGENETICS.value_counts()

CYTOGENETICS
Normal                                                       752
46,XX                                                         62
46,XY[21]                                                     55
46,XY                                                         54
46,XY[20]                                                     40
                                                            ... 
46,XX {7]                                                      1
46XY,+1,der(1;7)(q10;p10), idem, +8                            1
46,XX,t(4;12)(q12;p13)[3]/46,idem,del(7)(q?)[12]/46,XX[1]      1
del(13)(q12q22)                                                1
46, XY, del(20)(q1?) [5]/46, XY [15]                           1
Name: count, Length: 708, dtype: int64

In [54]:
def extract_sex(cytogenetics):
    if pd.isna(cytogenetics):
        return 2
    elif "46,XX" in cytogenetics:
        return 0
    elif "46,XY" in cytogenetics:
        return 1
    else:
        return 2

In [55]:
def parse_cytogenetics(cyto):
    cyto = str(cyto).lower()  # Convertir en string et mettre en minuscule

    # 1 Extraire le nombre de chromosomes
    chrom_match = re.search(r"\b(40|41|42|43|44|45|46|47|48|49|50)\b", cyto)
    num_chromosomes = int(chrom_match.group()) if chrom_match else None

    # 2️ Utiliser la fonction existante pour déterminer le sexe
    sexe = extract_sex(cyto)  # Remplacement par la fonction existante

    # 3️ Extraire le nombre d’échantillons testés
    sample_match = re.search(r"\[(\d+)\]", cyto)
    num_samples = int(sample_match.group(1)) if sample_match else None

    # 4️ Détection des anomalies (nouvelle logique)
    has_anomaly = 0  # Par défaut, pas d’anomalie
    if num_chromosomes and num_chromosomes != 46:
        has_anomaly = 1
    if re.match(r"^(46,xy(\[\d+\])?|46,xx(\[\d+\])?)$", cyto) is None:
        has_anomaly = 1

    return pd.Series([num_chromosomes, sexe, num_samples, has_anomaly])

# Appliquer la fonction au DataFrame (assurez-vous que 'train_set_clin' contient une colonne 'CYTOGENETICS')
Xtest_clin[["Nombre_Chromosomes", "SEX", "Nb_Echantillons", "Anomalies"]] = Xtrain_clin["CYTOGENETICS"].apply(parse_cytogenetics)

In [56]:
data_train["SEX"] = data_train["CYTOGENETICS"].apply(extract_sex)
print(data_train["SEX"].value_counts())

SEX
2    1623
1    1146
0     459
Name: count, dtype: int64


In [57]:
data_train[data_train["SEX"]==1]

,ID,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH,SEX
8,KYW1000,16.0,0.66,0.0891,0.0033,10.4,157.0,"47,XY,+11[19]/46,XY[1]",4,106156669.0,106156669.0,G,T,TET2,p.E524X,stop_gained,0.376,431.0,1
9,KYW1000,16.0,0.66,0.0891,0.0033,10.4,157.0,"47,XY,+11[19]/46,XY[1]",4,106158407.0,106158407.0,-,T,TET2,p.N1103fs,frameshift_variant,0.056,248.0,1
10,KYW1000,16.0,0.66,0.0891,0.0033,10.4,157.0,"47,XY,+11[19]/46,XY[1]",4,106196458.0,106196458.0,C,-,TET2,p.F1597fs,frameshift_variant,0.316,718.0,1
11,KYW1000,16.0,0.66,0.0891,0.0033,10.4,157.0,"47,XY,+11[19]/46,XY[1]",X,15809094.0,15809094.0,C,T,ZRSR2,p.R27X,stop_gained,0.833,54.0,1
12,KYW1000,16.0,0.66,0.0891,0.0033,10.4,157.0,"47,XY,+11[19]/46,XY[1]",X,123220400.0,123220400.0,T,A,STAG2,p.Y1019X,stop_gained,0.605,119.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,KYW999,1.0,1.16,0.2900,0.0928,6.3,233.0,"46,XY,inv(17)(p11.2q21.2)[1]/46,XY[19]",20,31022412.0,31022412.0,-,A,ASXL1,p.H633fs,frameshift_variant,0.471,410.0,1
3224,KYW999,1.0,1.16,0.2900,0.0928,6.3,233.0,"46,XY,inv(17)(p11.2q21.2)[1]/46,XY[19]",21,36252866.0,36252866.0,G,C,RUNX1,p.R166G,non_synonymous_codon,0.436,330.0,1
3225,KYW999,1.0,1.16,0.2900,0.0928,6.3,233.0,"46,XY,inv(17)(p11.2q21.2)[1]/46,XY[19]",X,15818077.0,15818077.0,G,A,ZRSR2,c.203+1G>A,nan,0.969,64.0,1
3226,KYW999,1.0,1.16,0.2900,0.0928,6.3,233.0,"46,XY,inv(17)(p11.2q21.2)[1]/46,XY[19]",X,123215272.0,123215272.0,A,-,STAG2,p.R940fs,frameshift_variant,0.854,205.0,1


In [58]:
#Traitement valeurs manquantes si homme femme ou unknown imputer mediane

cols_to_impute = ['BM_BLAST', 'HB', 'PLT', 'ANC', 'WBC', 'MONOCYTES', 'DEPTH', 'VAF']

# Fonction pour imputer en utilisant la médiane par groupe
def impute_by_group(df, group_col, cols):
    imputed_df = df.copy()
    for group in df[group_col].unique():
        group_median = df[df[group_col] == group][cols].median()  
        imputed_df.loc[df[group_col] == group, cols] = df[df[group_col] == group][cols].fillna(group_median)
    return imputed_df

data_train = impute_by_group(data_train, "SEX", cols_to_impute)

In [59]:
missing_ratio = data_train.isna().mean() 
cols_to_drop = missing_ratio[missing_ratio > 0.5].index.tolist()
data_train = data_train.drop(columns = cols_to_drop)


Variables non utile pour le modèle :

-  CHR, START, END : La position seule n’indique pas l’impact biologique de la mutation. Ce qui est vraiment utile, c’est le gène affecté et l’effet de la mutation (déjà fourni par les variables GENE et EFFECT).
Ce sont des variables très spécifiques qui risquent d’être trop granulaires pour un modèle de ML (car des milliers de positions peuvent exister). On pourrait toutefois les regrouper en secteurs.

- CENTER : Le centre médical n'est pas pertinente pour répondre à la problématique : évaluer le risque de décès pour les patients diagnostiqués avec un cancer du sang.
On peut toutefois vérifier s'il existe une différence significatives concernant les compétences/qualités des cliniques, qui impacteraient le risque de décès.

Variables utiles  :
Ces variables donnent des informations sur l’état physiologique du patient et peuvent être de bons indicateurs du risque de décès.

* BM_BLAST : Indicateur clé de la proportion de cellules sanguines anormales dans la moelle osseuse. Un taux élevé est souvent associé à une maladie plus agressive et un pronostic plus défavorable.

* WBC (Globules blancs) : Peut refléter la gravité de la maladie ; une élévation anormale peut être associée à un risque accru de complications.

* ANC (Neutrophiles absolus) : Essentiel pour évaluer l'état du système immunitaire et les risques d’infections opportunistes.

* MONOCYTES : Un taux élevé de monocytes peut être associé à une inflammation chronique et à une progression plus rapide de la maladie.

* HB (Hémoglobine) : Un faible taux d’hémoglobine est un marqueur d’anémie, souvent observé dans les cancers du sang avancés.

* PLT (Plaquettes) : Une thrombocytopénie (faible nombre de plaquettes) peut être associée à un mauvais pronostic, car elle peut entraîner des complications hémorragiques.

* CYTOGENETICS : Les anomalies chromosomiques sont souvent des indicateurs de l’agressivité du cancer. Par exemple, la monosomie 7 est un marqueur de haut risque.

On vérifie la granularité des colonnes (confirmer la supression des colonnes de positions de la mutations):
- START à 4645 valeurs distinctes
- END a 4664 vlaures disctinctes
- CHR possède 23 données.

On remarque également qu'il n'y a que 3'026 ID distincts pour plus de 10'000 lignes dans le dateset molecular_train, ce qui signifie que certains patient ont plusieurs mutations génétiques.

Il peut donc être pertinent de synthétiser les informations de ce dataset pour ne posséder qu'une ligne par ID. En effet, il ne faut pas faire une analyse par mutation mais une analyse par patient.

On peut créer une variable qui stock le nombre total de mutations par patients, cela pourrait améliorer la mesure de risque. 



In [60]:
data_train.nunique()

ID                1193
BM_BLAST            82
WBC                458
ANC                937
MONOCYTES          271
HB                 119
PLT                322
CYTOGENETICS       708
CHR                 22
START             1534
END               1528
REF                140
ALT                 81
GENE                83
PROTEIN_CHANGE    1589
EFFECT               9
VAF                792
DEPTH             1041
SEX                  3
dtype: int64

CYTOGENETICS : Les anomalies chromosomiques sont souvent des indicateurs de l’agressivité du cancer. Par exemple, la monosomie 7 est un marqueur de haut risque.

Or il y'a plus de 1194 données différents pour 2936 valeurs, (3223-2936 == NA).

In [61]:
data_train["ID"].value_counts()

ID
KYW791     14
KYW203     13
KYW733     13
KYW201     13
KYW202     11
           ..
KYW696      1
KYW1099     1
KYW460      1
KYW692      1
KYW194      1
Name: count, Length: 1193, dtype: int64

In [62]:
Xtrain_clin["ID"].value_counts()
Xtrain_mol["ID"].value_counts() #dans mol il y a des id qui se répètent

ID
KYW791     14
KYW733     13
KYW201     13
KYW203     13
KYW202     11
           ..
KYW1083     1
KYW35       1
KYW170      1
KYW974      1
KYW18       1
Name: count, Length: 1054, dtype: int64

In [68]:
target_data["ID"].value_counts()

ID
P132697    1
P120833    1
P120866    1
P120852    1
P120858    1
          ..
P110864    1
P110865    1
P110866    1
P110867    1
P121834    1
Name: count, Length: 3173, dtype: int64

In [69]:
# Sélection des colonnes numériques
numerical_cols = data_train.select_dtypes(include=['float64', 'int64']).columns

In [70]:
# Normalisation et encodage
scaler = StandardScaler()
Xtrain_scaled = scaler.fit_transform(data_train[numerical_cols])


In [ ]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

In [72]:
# Reconstruire un DataFrame avec les données normalisées
data_train_scaled_df = pd.DataFrame(Xtrain_scaled, columns=numerical_cols)

In [73]:
data_train_non_numeric = data_train.drop(columns=numerical_cols)

In [77]:
# Concaténation des nouvelles colonnes normalisées avec les non numériques
data_train_complete = pd.concat([data_train_non_numeric, data_train_scaled_df], axis=1)

On veut rattacher les deux dataframe pour n'en créer qu'un seul, on utilisera les clés uniques : "ID".
Dans le dataframe molecule train, on a plus de dis milles lignes. Alors qu'il n'yen a que 3323 dans le dataframe clinical train.
Celles dont l'ID n'est pas inclus dans clinical train seront supprimés.

À traiter : 
• Duplicated observations

• Missing data

• Heterogeneity in data reporting conventions: frequency of observations, units, date format, missing data representation, ... 

• Inconsistencies between databases

• ...

Some rules:

• Get acquainted with the data to get a better understanding of what they represent and understand the way they are encoded


• Perform exploratory analyses: number of observations, number of  missing values, the average value of features, median, standard deviation, inter-quartile range, binning/clustering correlations, ...

• Objective: identification of useful features, outliers, ...

Data cleaning :

• Missing data handling: deletion or imputation

• Duplicated data: techniques for deduplicating data

• Handling of outliers: deletion, winsorization

• Inconsistencies: partially automatic detection is made possible through functional dependencies (e.g. a city must always have the same zip code)

Data transformation :

• Feature scaling: some ML algorithms do not perform well when the input numerical attributes display very different scales, e.g. revenue vs. ratio. Normalization and standardization are two common ways to get all attributes to have the same scale Aggregation: e.g. cumulate daily data to transform them into monthly data

• Features construction: addition or replacement of some features by new features computed from existing ones (e.g. log transformation or power transformation to capture non-linearities)

• Discretization: the transformation of numerical values into classes

In [ ]:
def prepare_survival_data(X, Y):
    Y['event'] = Y['event'].astype(bool)
    Y_surv = Surv.from_dataframe("event", "time", Y)
    return X, Y_surv

In [ ]:
X_train, Y_train_surv = prepare_survival_data(X_train, Y_train)
X_test, Y_test_surv = prepare_survival_data(X_test, Y_test)

In [ ]:
#séparer les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#imputer = SimpleImputer(strategy="median")
#X_train[['BM_BLAST', 'HB', 'PLT']] = imputer.fit_transform(X_train[['BM_BLAST', 'HB', 'PLT']])
#X_test[['BM_BLAST', 'HB', 'PLT']] = imputer.transform(X_test[['BM_BLAST', 'HB', 'PLT']])

In [79]:
# 1. Modèle de Cox
#cox_model = CoxPHSurvivalAnalysis()
#cox_model.fit(X_train, Y_train_surv)
#cox_cindex = cox_model.score(X_test, Y_test_surv)
#print(f"Concordance Index - Cox Model: {cox_cindex:.4f}")

In [80]:
# 2. Random Survival Forest
#rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, random_state=42)
#rsf.fit(X_train, Y_train_surv)
#rsf_cindex = rsf.score(X_test, Y_test_surv)
#print(f"Concordance Index - RSF: {rsf_cindex:.4f}")

In [81]:
# 3. LightGBM (approche survie)
#lgb_train = lgb.Dataset(X_train, label=Y_train["time"], free_raw_data=False)
#lgb_test = lgb.Dataset(X_test, label=Y_test["time"], free_raw_data=False)
#params = {"objective": "survival", "metric": "cindex", "boosting_type": "gbdt", "learning_rate": 0.05}
#lgb_model = lgb.train(params, lgb_train, num_boost_round=100)
#lgb_preds = lgb_model.predict(X_test)
#lgb_cindex = np.corrcoef(lgb_preds, Y_test["time"])[0, 1]
#print(f"Concordance Index - LightGBM: {lgb_cindex:.4f}")

In [82]:
# 4. DeepSurv (PyCox)
#deep_surv = CoxPH(tt.optim.Adam(0.01), X_train.shape[1])
#deep_surv.fit(X_train, Y_train_surv, batch_size=32, epochs=100, verbose=True)
#deep_cindex = deep_surv.score(X_test, Y_test_surv)
#print(f"Concordance Index - DeepSurv: {deep_cindex:.4f}")